# Intro
I dette project vil jeg lave et objectorienteret ConvNet, der skal kunne genkende forskellige typer af tøj og sortere dem i kategorier.
Jeg har valgt at lave dem objektorienteret, da jeg før har forsøgt at lave ANN'er, hvor det var baseret hovedsagligt ud fra funktioner, hvilket ledte til, at de hele blev meget uoverskueligt og repeterivt.

# Importering af biblioteker
Først vil jeg importere nogle biblioteker med foruddefinerede funktioner, der kan hjælpe med at gøre udviklingsprocessen nemmere.

In [543]:
import numpy as np # Matricer og vektorer
np.random.seed(0) # For at kunne genskabe de samme resultater.

from scipy import signal as signal # Til Fast Fourier Transformation Convolution.

# Defineringen af et lag
Først vil jeg definere en superklasse, lag, som de andre typer af lag skal nedarve fra.

In [532]:
class Lag:
    # __init__, også kendt som constructor, er kort for 'intialize', og 
    # er en metode, der kaldes, når et objekt af klassen Lag oprettes. 
    # Init har ikke noget output.
    def __init__(self) -> None:
        # Input- of outputtypen afhænger af, hvilekt slags lag det er. 
        # Men alle lag har in og out, så jeg definerer dem som None her, 
        # for at de er der, når jeg nedarver herfra.
        self.input = None
        self.output = None
    
    # I frem-metoden, udregner vi outputtet ud fra inputtet. Hvad der sker 
    # i frem(), afhænger af, hvilket slags lag det er.
    def frem(self, input):
        pass

    # I tilbage-metoden, udregner vi hvilke ændringer, der skal foretages 
    # på de trænede vægte og biaser, ud fra outputtet.
    def tilbage(self, pa_output): 
        # 'pa_output' er 'partielle afledte af outputtet'. Her snakkes der
        # om outputtet som blev udregnet i frem(). Dette er gives til laget,
        # fra det næste lag i netværkert under bagudpropagering.
        pass

    def update(self, skridt_længde):
        # Denne metode opdaterer vægtene og biaserne i laget, ud fra de 
        # ændringer, der blev udregnet i tilbage(). 

        # 'skridt_længde' er en hyperparameter, der bestemmer, hvor meget
        # vi vil ændre på vægtene og biaserne. Denne værdi er fastsat af mig
        # i dette tilfælde. Man kunne dog vælge at få den til at variere, alt
        # efter hvor langt netværket er fra sit mål.
        pass

# Definering af FF_Lag
Et FF-lag, eller et 'Fuldt forbundet' lag, er en type af lag, der ofte findes i Artificial Neural Networks. Her består et lag af neuroner, der er forbundet med synapser til alle de kommende lag. Her undlader jeg dog at bruge en aktiveringsfunktion, da dette kan gøres i et separat lag.

In [533]:
# Dette er et fuldt forbundet lag, også kendt som et fully connected layer, eller dense layer.
class FF(Lag):
    def __init__(self, n_input : int, n_output : int) -> None:
        # Her er n_input antallet af neuroner i det forrige lag, og n_output
        # er antallet af neuroner i det næste lag.
        # Dett skal jeg bruge for at vide, hvilket facon vægte og biaser skal have.
        
        self.w = np.random.randn(n_output, n_input).astype(np.float64)
        # w er vægtene. De er initialiseret til at være tilfældige tal. Vægte er
        # opbygget til at være samlet i en matrix, hvor højde er antaller af
        # neuroner i det næste lag, og bredde er antallet af neuroner i det forrige lag.
        # Vægte bliver ganget på inputtet i frem() metoden.
        
        self.b = np.random.randn(n_output, 1).astype(np.float64)
        # b er biaserne. De er initialiseret til at være tilfældige tal. Biaser er
        # opbygget til at være en vektor, hvor højden er antallet af neuroner
        # i det næste lag.
        # Biaser bliver lagt til vægtene i frem() metoden.

        self.pa_w = np.zeros_like(self.w).astype(np.float64)
        self.pa_b = np.zeros_like(self.b).astype(np.float64)
        # Disse variabler er sat til None, da de først bliver brugt i tilbage() og update() metoden.

        self.iterations = 0
        pass
    
    def frem(self, input) -> np.ndarray:
        # Normalt ville man have en aktiveringsfunktion her, men dette kan også gøres
        # som et separat lag. Derfor er det udeladt her for at gøre det mere fleksibelt.
        
        vægtet = np.dot(self.w, input)
        # Inputtet ganges med vægtene, for at udregne outputtet. Dette gøres ved at
        # lave en matrix-transformation af inputvektoren.

        forskudt = vægtet + self.b
        # Biasvektoren lægges til det vægtede input. Dette kan forskyde 

        self.input = input
        # Inputtet gemmes, så det kan bruges i tilbage() metoden.

        return forskudt

    def tilbage(self, pa_output) -> np.ndarray:
        
        self.pa_w += np.dot(pa_output, self.input.T)
        # Hvis man differencerer frem() metoden med hensyn til vægtene, finder man at
        # resultatet er inputtet ganget med partielle afledte af outputtet.
        # Vi transponerer inputtet, da det er en kolonnevektor, så resultatet bliver 
        # en matrix, der passer til vægtenes facon.

        self.pa_b += pa_output # * 1
        # Da biasene kun adderes på, er den partielle afledte af biaserne lig med 1.

        # Vi gemmer de partielle afledte, så de kan bruges i update() metoden. Grunden til at
        # vi ikke opdaterer vægtene og biaserne med det samme, er at vi gerne vil køre mini-batches.
        # Dette betyder at vi først vil køre en række input igennem, og derefter opdatere vægtene og biaserne.

        pa_input = np.dot(self.w.T, pa_output)
        # I denne metode har vi brugt pa_output fra det næste lag, til at udregne de ændringer,
        # vi skal lave til vægtene og biaserne. Vi skal også bruge et pa_output til det forrige lag.
        # Da inputtet ganges med vægtene, for at udregne outputtet, er den partielle afledte af inputtet
        # lig med vægtene ganget med pa_output fra det næste lag. Vi transponerer vægtene, for at få
        # de så matcher til outputtet.

        self.iterations += 1

        return pa_input
    
    def update(self, skridt_længde):
        self.w -= skridt_længde * (self.pa_w/self.iterations)
        self.b -= skridt_længde * (self.pa_b/self.iterations)
        # Vægtene og biaserne opdateres ved at trække den partielle afledte af vægtene
        # Da vi vil gå mod minimum, fremfor maksimum, trækker vi fra, og ganger
        # med skridtlængden.

        self.pa_w = np.zeros_like(self.w).astype(np.float64)
        self.pa_b = np.zeros_like(self.b).astype(np.float64)
        # Vi nulstiller de partielle afledte, så de er klar til næste Epoke.

        self.iterations = 0
        # Vi nulstiller antallet af iterationer, så vi kan tælle op til næste Epoke.
    

# Definering af Aktiveringsfunktionslag
Normal finder man aktiveringsfunktioner inde i neronerne, men man kan også vælge at lave dem til et separat lag. Så kan man holde kompleksiteten nede. Hellere flere simple lag, end få komplicerede.
<br>Desuden er det alstå også bare pænere at kigge på...

In [534]:
# Et lag, der bruger en aktiveringsfunktion. Dette lag har ingen vægte eller biaser.
class Funktion(Lag):
    def __init__(self, funktion, afledt_funktion) -> None:
        # Funktion er en aktiveringsfunktion, og afledt_funktion er sjovt nok den afledte funktionen.
        
        self.funktion = funktion
        self.afledt_funktion = afledt_funktion
        # Her gemmes funktionerne, så de kan bruges i frem() og tilbage() metoderne.
    
    def frem(self, input) -> np.ndarray:
        
        self.input = input
        # Inputtet gemmes, så det kan bruges i tilbage() metoden.

        ikke_lineært = self.funktion(input)
        # Funktionen anvendes på inputtet, og outputtet returneres. Aktiveringsfunktioner bliver brugt
        # til at give netværkets beslutningsbariere en ikke-lineæritet, da ellers ville kunne blive
        # reduceret til en lineær transformation.
        # Dette er vigtigt, da mange problemer ikke er lineært separable.

        return ikke_lineært
    
    def tilbage(self, pa_output) -> np.ndarray:
        # Da aktiveringsfunktioner ikke har nogle vægte eller biaser, er der ikke noget at opdatere.

        pa_input = np.multiply(pa_output, self.afledt_funktion(self.input))
        # Her tager vi Hadamard-produktet af pa_output og den afledte funktion af inputtet.

        return pa_input
        pass

In [535]:
# For specifikke aktiveringsfunktioner, kan vi nedarve fra Funktion, og give dem en specifik funktion og afledt funktion.
# Her bruger jeg sigmoid, da den giver nogle pæne værdier mellem 0 og 1. Den er meget ligesom en tanH-funktion, der giver
# værdier mellem -1 og 1. Dog kan jeg godt lide den pæne form, som sigmoid har.
class Sigmoid(Funktion):
    def __init__(self):

        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        afledt_sigmoid = lambda x: sigmoid(x) * (1 - sigmoid(x)) 
        # Her bruger jeg funktionelle lambda-udtryk, for at definere sigmoid og dens afledte funktion.
        # Dette gør, at jeg kan gemme dem i en variabel, og give dem til super's constructor.
        # Lamda-udtryk har det some regel også med at være lidt hurtigere end funktioner, men det
        # betyder ikke meget her, fordi jeg ikke fokuserer på hastighed.

        # I den afledte funktion, bruger jeg sigmoid(x) for at undgå at skulle udregne med for store tal.

        super().__init__(sigmoid, afledt_sigmoid)

# Udregning af netværkets tab
Når vi skal træne det neurale netværk, har vi brug for at vide, hvor godt netværket klarer sig. Derfor skal vi bruge et udtryk, der hedder 'tab'. Dette kan man finde ved at sammenligne netværkets output med éns ønskede output.
$$
Tab = 
(Output - ØnsketOutput)^2
$$
Grunden til at man sætter det i anden, er fordi programmet går efter mindst muligt tab. Hvis denne værdi kunne gå i negativ, fordi det ønskede output var højere end outputtet, ville programmet blive ved med at bevæge outputtet længere væk fra det ønskede output, fordi det ville blive en lavere værdi. Dertil får man den bonus, at tabet bliver eksponintielt voldsommere, jo mere forkert outputtet er.

In [536]:
# Ikke brugbar til træning, da man kun skal bruge den afledte funktion til at køre bagudpropageringen.
# Dog kan man bruge denne her til at holde øje med, om netværket lærer noget.
def tab(output, ønsket_output) -> float:
    if output == np.nan: print("output er nan")
    if ønsket_output == np.nan: print("ønsket_output er nan")
    # Tab er en funktion, der udregner hvor langt netværket er fra sit mål.
    return np.sum((output - ønsket_output) ** 2)

def pa_tab(output, ønsket_output) -> np.ndarray:
    # Den partielle afledte af tab-funktionen med hensyn til outputtet.
    return 2 * (output - ønsket_output)

# Definering af et Netværk
Normalt ville man ikek behøve at definere en klasse til at køre alle udregningerne, men da jeg gerne vi kunne lave flere versioner af forskellige netværk til tests med mere, er det hurtigere at samle det til en klasse, så jeg kan bruge den på forskellige måder, uden at skulle genskrive al koden.

In [537]:
class Netværk():
    def __init__(self, lag) -> None:
        self.lag = lag
        # Lag er en liste af lag, som netværket består af.

    def frem(self, input) -> np.ndarray:
        # Frem-metoden tager inputtet, og sender det igennem alle lagene i netværket.

        for l in self.lag:
            input = l.frem(input)
            # Inputtet sendes igennem hvert lag, og outputtet bliver det nye input.
        return input
    
    def __tilbage(self, pa_output) -> np.ndarray:
        # Tilbage-metoden tager partielle afledte af outputtet, og sender dem igennem alle lagene i netværket.

        for l in reversed(self.lag):
            pa_output = l.tilbage(pa_output)
            # Partielle afledte af outputtet sendes igennem hvert lag, og outputtet bliver det nye input.
        return pa_output
    
    def __update(self, skridt_længde) -> None:
        # Update-metoden opdaterer vægtene og biaserne i alle lagene i netværket.

        for l in self.lag:
            l.update(skridt_længde)
            # Vægtene og biaserne i hvert lag opdateres.
    
    def __tab(self, output, ønsket_output) -> float:
        # Tab-metoden tager outputtet og ønsket_outputtet, og udregner tabet.
        # Dette er udelukkende, så vi kan se, hvor langt netværket er fra sit mål.
        return tab(output, ønsket_output)
    

    # Jeg har valgt at lave de overstående metoder private, da de ikke skal bruges udenfor klassen.


    def træn(self, inputs : np.ndarray, ønskede_outputs : np.ndarray, skridt_længde : float, epoker : int, batchstørrelse : int, tab_print_interval : int) -> None:
        # Træn-metoden tager inputs, ønskede_outputs, skridt_længde, antallet af epoker og batchstørrelsen.
        # Den træner netværket, ved at køre inputs igennem netværket, udregne tabet, og køre bagudpropagering.
        # Dette gøres for et antal epoker, og med en batchstørrelse.
        
        # Mini-batch gradientnedstigning.
        for _ in range(epoker): 
            # For hver epoke

            for i in range(batchstørrelse): 
                # For hver batch

                output = self.frem(inputs[i])
                # Inputtet sendes igennem netværket, og outputtet returneres.

                pa_output = pa_tab(output, ønskede_outputs[i])
                # Den partielle afledte af tabet for outputtet udregnes.

                self.__tilbage(pa_output)
                # Partielle afledte sendes igennem netværket.
            
            self.__update(skridt_længde)
            # Efter at have kørt en batch igennem, opdateres vægtene og biaserne.

            if _ % tab_print_interval == 0:
                print(self.__tab(output, ønskede_outputs[i]))
                # Her printes tabet, så vi kan se, om netværket lærer noget.
        
        print("\nTræning færdig")
        print(f"Afsluttet med tab: {self.__tab(output, ønskede_outputs[i])}")


# Test af ANN
Al den overstående kode er nok til at definere et ANN, så før jeg bevæger mig videre til at tilbygge Conv-lag på ANN'et, vil jeg lige teste om det fungerer. Til dette bruger jeg den klassiske XOR test, der er ikke-lineært separabelt problem.

In [538]:
ann = Netværk([
    FF(2, 3), 
    Sigmoid(), 
    FF(3, 1), 
    Sigmoid()
    ])
# Her opretter jeg et netværk, som består af et inputlag med 2 neuroner, et skjult lag med 3 neuroner,
# et outputlag med 1 neuron.

inputs = np.reshape([[0, 0], [0, 1], [1, 0], [1, 1]], (4, 2, 1))
ønskede_outputs = np.reshape([[0], [1], [1], [0]], (4, 1, 1))
# Her opretter jeg inputs og ønskede_outputs, som er XOR-gate. Dette er et klassisk problem, som

ann.træn(inputs, ønskede_outputs, 0.1, 10000, 4, 500)


0.8247279374202696
0.2486813344952613
0.2510344233978957
0.24649596383124286
0.2298455072475089
0.193482614863555
0.13838987454624507
0.09095274070378688
0.061116828953222326
0.04352230782837874
0.032768929922133
0.025807042516000685
0.02104697796010895
0.017638222838066007
0.01510223579313258
0.013155410790473736
0.011621523406481597
0.010386432707862384
0.009373500621181394
0.008529640573789299

Træning færdig
Afsluttet med tab: 0.007818383845293847


Her man man se, at vi har opnået et relativt lavt tab. Nu vil jeg prøve at køre den igennem for at se dets resultater.

In [542]:
print(f"[0,0] : Expected 0 : Output {np.round(ann.frem(np.array([[0], [0]]))[0,0])}")
print(f"[1,0] : Expected 1 : Output {np.round(ann.frem(np.array([[1], [0]]))[0,0])}")
print(f"[0,1] : Expected 1 : Output {np.round(ann.frem(np.array([[0], [1]]))[0,0])}")
print(f"[1,1] : Expected 0 : Output {np.round(ann.frem(np.array([[1], [1]]))[0,0])}")

[0,0] : Expected 0 : Output 0.0
[1,0] : Expected 1 : Output 1.0
[0,1] : Expected 1 : Output 1.0
[1,1] : Expected 0 : Output 0.0


Den svarer rigtigt på opgaven.

# Tid til ConvNet
Men når man har med billedgenkendelse at gøre, er det en god idé at montere et ConvNet i starten af sit ArtNet. Så det er det jeg har tænkt mig at gøre nu. Det første jeg vil gøre, er at definere et ConvLag.

In [544]:
class Conv(Lag):
    def __init__(self, input_facon : tuple, kernel_størrelse: int, dybde : int) -> None:
        
        input_dybde, input_højde, input_bredde = input_facon
        # Her definerer jeg forskellgie variabler til at beskrive input faconen. Detter er blot for
        # at gøre det nemmere at læse koden.

        self.dybde = dybde
        # Antal af kernels.

        self.input_facon = input_facon
        self.input_dybde = input_dybde
        # Billeder har ofte en dybde, som beskriver hvor mange kanaler de har. Dette kunne være RGB, som
        # har 3 kanaler, eller sort-hvid, som kun har 1 kanal.

        self.output_facon = (dybde, input_højde - kernel_størrelse + 1, input_bredde - kernel_størrelse + 1)
        # Der er lige så mange output som der er kernels.

        self.kernel_facon = (dybde, input_dybde, kernel_størrelse, kernel_størrelse)
        self.kernels = np.random.randn(*self.kernel_facon).astype(np.float64)
        self.biases = np.random.randn(*self.output_facon).astype(np.float64)
        # Her definerer jeg kernels som en 4D tensor. Men det kan være til fordel at forestille
        # sig dem som flere 3D tensorer, der har hvert sit output.

        
        self.pa_kernels = np.zeros(self.kernel_facon).astype(np.float64)
        self.pa_biases = np.zeros(self.output_facon).astype(np.float64)
        # Disse variabler er sat til None, da de først bliver brugt i tilbage() og update() metoden.
        
        self.iterations = 0

    def frem(self, input) -> np.ndarray:
        
        self.input = input

        self.output = np.copy(self.biases)
        # Her kopierer jeg biasene, så jeg ikke behøver at addere dem senere.

        for i in range(self.dybde):
            for j in range(self.input_dybde):
                self.output[i] += signal.correlate2d(input[j], self.kernels[i, j], mode='valid')
        # Her bruger jeg scipy's fftconvolve funktion, som er hurtigere end numpy's convolve funktion.
        # Den bruger Fast Fourier Transformation til at udregne konvolutionen, hvilket er hurtigere end
        # at gøre det direkte. Eller... Det er faktisk ikke konvolution. Det er korrelation, men det er
        # næsten det samme.
        # Jeg bruger mode='valid', da jeg ikke vil have padding. Dette betyder at outputtet bliver mindre
        # end inputtet.

        return self.output
    
    def tilbage(self, pa_output) -> np.ndarray:
        
        pa_kernels = np.zeros(self.kernel_facon).astype(np.float64)
        pa_input = np.zeros(self.input_facon).astype(np.float64)

        for i in range(self.dybde):
            for j in range(self.input_dybde):
                pa_kernels[i, j] = signal.correlate2d(self.input[j], pa_output[i], mode='valid')
                # Når man skal udregne de partielle afledte af kernels, kan man forkorte det til at være
                # inputtet korreleret med de partielle afledte af outputtet.

                pa_input[j] += signal.convolve2d(pa_output[i], self.kernels[i, j], mode='full')
                # For at udregne de partielle afledte af inputtet, kan man bruge de partielle afledte af outputtet
                # konvolveret med kernels.

        # Her udregner jeg de partielle afledte af kernels og inputtet.

        self.pa_kernels = pa_kernels
        self.pa_biases = pa_output
        # Jeg gemmer de partielle afledte, så de kan bruges i update metoden.

        return pa_input
    
    def update(self, skridt_længde):
        self.kernels -= skridt_længde * (self.pa_kernels/self.iterations)
        self.biases -= skridt_længde * (self.pa_biases/self.iterations)
        # Vægtene og biaserne opdateres ved at trække den partielle afledte af vægtene
        # Da vi vil gå mod minimum, fremfor maksimum, trækker vi fra, og ganger
        # med skridtlængden.

        self.pa_kernels = np.zeros(self.kernel_facon).astype(np.float64)
        self.pa_biases = np.zeros(self.output_facon).astype(np.float64)
        # Vi nulstiller de partielle afledte, så de er klar til næste Epoke.

        self.iterations = 0
        # Vi nulstiller antallet af iterationer, så vi kan tælle op til næste Epoke.

# Definering af formateringslag
Et ConvNets lag er ofte multidimensionellen, mens et ArtNets her det med at være 1D. Derfor skal vi omformatere faconen fra ConvNettet til ArtNettet, så de kan snakke sammen. NumPy har heldigvis en funktion der kan gøre det hele for mig.

In [ ]:
class Formatering(Lag):
    def __init__(self, input_facon : tuple, output_facon : tuple) -> None:
        self.input_facon = input_facon
        self.output_facon = output_facon
        pass

    def frem(self, input : np.ndarray) -> np.ndarray:
        return np.reshape(input, self.output_facon)
    
    def tilbage(self, pa_output : np.ndarray) -> np.ndarray:
        return np.reshape(pa_output, self.input_facon)

# Test af ConvNet og ArtNet
Nu skal vi se om det hele spiller sammen. Jeg vil teste det på [dette datasæt](https://www.kaggle.com/datasets/agrigorev/clothing-dataset-full). Jeg vil vælge 10 af kategorierne derfra.

In [ ]:
cnn = Netværk([
    Conv((1, 28, 28), 3, 5),
    Sigmoid(),
    Formatering((3, 24, 24), (3*24*24, 1)),
    FF(3*24*24, 16),
    Sigmoid(),
    FF(16, 10),
    Sigmoid()
    ])